In [33]:
import cv2
import urllib
import numpy as np
import os
import glob

from urllib.request import urlopen

#check version
print("OpenCv version: {}", cv2.__version__)

OpenCv version: {} 4.5.3


In [34]:
CHECKERBOARD = (6,9)
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_CHECK_COND+cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

In [35]:
images = glob.glob('./image/checker_*.jpg')
for fname in images:
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("Found " + str(N_OK) + " valid images for calibration")
print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")

Found 8 valid images for calibration
DIM=(2592, 1944)
K=np.array([[1238.8973384191427, 0.0, 1296.7584921501195], [0.0, 1239.1028827686082, 976.3825196094416], [0.0, 0.0, 1.0]])
D=np.array([[-0.1307433594261936], [0.04581714762639465], [-0.13650221908717208], [0.09875685944472529]])


In [46]:
# You should replace these 3 lines with the output in calibration step
DIM=(2592, 1944)
K=np.array([[1238.8973384191427, 0.0, 1296.7584921501195], [0.0, 1239.1028827686082, 976.3825196094416], [0.0, 0.0, 1.0]])
D=np.array([[-0.1307433594261936], [0.04581714762639465], [-0.13650221908717208], [0.09875685944472529]])
#def undistort():
img = cv2.imread('./image/checker_00.jpg')
h,w = img.shape[:2]
map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

print(img.shape)
print(undistorted_img.shape)

img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_AREA)
undistorted_img = cv2.resize(undistorted_img, (512, 512), interpolation=cv2.INTER_AREA)

cv2.imshow("original", img)
cv2.imshow("undistorted", undistorted_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(1944, 2592, 3)
(1944, 2592, 3)


In [45]:
#undistort()